# Import Libraries & Data

In [1]:
import os
import glob
import re
import numpy as np
import pandas as pd

In [2]:
dataset = pd.read_pickle('kaggle_openface_merged.pkl')

In [3]:
dataset

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,file_name,label,video_id
0,1,0,0.000,0.98,1,-0.265343,0.270043,-0.925565,-0.394123,0.281717,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,trial_lie_001.csv,lie,trial_lie_001
1,2,0,0.033,0.98,1,-0.269314,0.280146,-0.921405,-0.399485,0.291082,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,trial_lie_001.csv,lie,trial_lie_001
2,3,0,0.067,0.98,1,-0.276762,0.281786,-0.918694,-0.402257,0.294677,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,trial_lie_001.csv,lie,trial_lie_001
3,4,0,0.100,0.98,1,-0.267571,0.284353,-0.920624,-0.390941,0.297649,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,trial_lie_001.csv,lie,trial_lie_001
4,5,0,0.133,0.98,1,-0.251324,0.286230,-0.924613,-0.374932,0.299439,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,trial_lie_001.csv,lie,trial_lie_001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91331,536,0,19.815,0.98,1,0.399038,0.150437,-0.904509,0.234647,0.184356,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,trial_truth_060.csv,truth,trial_truth_060
91332,537,0,19.852,0.98,1,0.394478,0.167984,-0.903420,0.232950,0.200715,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,trial_truth_060.csv,truth,trial_truth_060
91333,538,0,19.889,0.98,1,0.392632,0.175124,-0.902869,0.232472,0.207146,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,trial_truth_060.csv,truth,trial_truth_060
91334,539,0,19.926,0.98,1,0.402144,0.140985,-0.904657,0.236606,0.176743,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,trial_truth_060.csv,truth,trial_truth_060


# Aggregate all data again

In [4]:
# at first we used only the AU column -> no good results so we added features for pose and gaze as well
#dataset.columns.values

In [5]:
dataset.sort_values(['video_id', 'face_id', 'timestamp'], inplace = True)

In [6]:
# keep only au, pose & gaze columns

au_cols = [c for c in dataset.columns if c.endswith("_r")]

pose_cols = [c for c in dataset.columns if c.startswith("pose_")]

gaze_cols = [c for c in dataset.columns if c.startswith("gaze_")]

final_columns = au_cols + pose_cols + pose_cols
#final_columns

## Compute aggregates (mean/max/std) per video

In [7]:
# compute aggregates (mean/max/std) per video pt.1

# compute  mean per video
video_mean = (dataset.groupby('video_id')[final_columns].mean())
video_mean

# compute AU mean per video
video_max = (dataset.groupby('video_id')[final_columns].max())
video_max

# compute AU mean per video
video_std = (dataset.groupby('video_id')[final_columns].std())
video_std

,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,AU09_r,AU10_r,AU12_r,AU14_r,...,pose_Tz,pose_Rx,pose_Ry,pose_Rz,pose_Tx,pose_Ty,pose_Tz,pose_Rx,pose_Ry,pose_Rz
video_id,,,,,,,,,,,,,,,,,,,,,
trial_lie_001,0.324179,0.163841,0.335093,0.071755,0.235783,0.411961,0.135490,0.242846,0.176749,0.029797,...,11.688945,0.051265,0.086833,0.080215,9.448078,6.835260,11.688945,0.051265,0.086833,0.080215
trial_lie_002,0.280204,0.265914,0.464621,0.173493,0.312652,0.378970,0.174106,0.391662,0.226734,0.224342,...,12.571943,0.070206,0.118626,0.082763,15.217302,8.960027,12.571943,0.070206,0.118626,0.082763
trial_lie_003,0.295436,0.208954,0.337934,0.073683,0.180666,0.449050,0.115039,0.249663,0.166174,0.032996,...,9.850529,0.016279,0.084538,0.042073,9.785724,2.380973,9.850529,0.016279,0.084538,0.042073
trial_lie_004,0.226205,0.188889,0.437078,0.101756,0.250258,0.350401,0.100998,0.290013,0.206197,0.029500,...,13.674829,0.047787,0.069979,0.047132,9.409074,5.050836,13.674829,0.047787,0.069979,0.047132
trial_lie_005,0.616221,0.462719,0.417614,0.198268,0.236141,0.390057,0.103947,0.388885,0.209974,0.021643,...,11.747484,0.056478,0.073683,0.074510,11.783181,6.163662,11.747484,0.056478,0.073683,0.074510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
trial_truth_056,0.155541,0.123977,0.255314,0.160047,0.198990,0.246462,0.096145,0.134633,0.075287,0.238906,...,2.142714,0.026197,0.023709,0.027037,4.569920,3.392376,2.142714,0.026197,0.023709,0.027037
trial_truth_057,0.248095,0.214293,0.441438,0.140062,0.121267,0.506605,0.127586,0.384440,0.206622,0.763449,...,56.802003,0.057541,0.231240,0.052074,17.765001,9.227349,56.802003,0.057541,0.231240,0.052074
trial_truth_058,0.216975,0.175674,0.392246,0.114391,0.222232,0.483808,0.107181,0.275670,0.184555,0.128719,...,9.149307,0.050380,0.053284,0.058234,6.114325,6.558754,9.149307,0.050380,0.053284,0.058234


In [8]:
# bring label column as well

data_temp = (dataset[['video_id', 'face_id', 'label']].drop_duplicates(subset = 'video_id'))

dataset_st_mean = video_mean.merge(data_temp, on = 'video_id')
dataset_st_mean

dataset_st_max = video_max.merge(data_temp, on = 'video_id')
dataset_st_max

dataset_st_std = video_std.merge(data_temp, on = 'video_id')
dataset_st_std

,video_id,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,AU09_r,AU10_r,AU12_r,...,pose_Ry,pose_Rz,pose_Tx,pose_Ty,pose_Tz,pose_Rx,pose_Ry,pose_Rz,face_id,label
0,trial_lie_001,0.324179,0.163841,0.335093,0.071755,0.235783,0.411961,0.135490,0.242846,0.176749,...,0.086833,0.080215,9.448078,6.835260,11.688945,0.051265,0.086833,0.080215,0,lie
1,trial_lie_002,0.280204,0.265914,0.464621,0.173493,0.312652,0.378970,0.174106,0.391662,0.226734,...,0.118626,0.082763,15.217302,8.960027,12.571943,0.070206,0.118626,0.082763,0,lie
2,trial_lie_003,0.295436,0.208954,0.337934,0.073683,0.180666,0.449050,0.115039,0.249663,0.166174,...,0.084538,0.042073,9.785724,2.380973,9.850529,0.016279,0.084538,0.042073,0,lie
3,trial_lie_004,0.226205,0.188889,0.437078,0.101756,0.250258,0.350401,0.100998,0.290013,0.206197,...,0.069979,0.047132,9.409074,5.050836,13.674829,0.047787,0.069979,0.047132,0,lie
4,trial_lie_005,0.616221,0.462719,0.417614,0.198268,0.236141,0.390057,0.103947,0.388885,0.209974,...,0.073683,0.074510,11.783181,6.163662,11.747484,0.056478,0.073683,0.074510,0,lie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,trial_truth_056,0.155541,0.123977,0.255314,0.160047,0.198990,0.246462,0.096145,0.134633,0.075287,...,0.023709,0.027037,4.569920,3.392376,2.142714,0.026197,0.023709,0.027037,0,truth
116,trial_truth_057,0.248095,0.214293,0.441438,0.140062,0.121267,0.506605,0.127586,0.384440,0.206622,...,0.231240,0.052074,17.765001,9.227349,56.802003,0.057541,0.231240,0.052074,0,truth
117,trial_truth_058,0.216975,0.175674,0.392246,0.114391,0.222232,0.483808,0.107181,0.275670,0.184555,...,0.053284,0.058234,6.114325,6.558754,9.149307,0.050380,0.053284,0.058234,0,truth
118,trial_truth_059,0.435807,0.236949,0.288428,0.175193,0.313232,0.399277,0.154920,0.325081,0.230249,...,0.120788,0.073406,12.692403,6.938594,7.485977,0.051070,0.120788,0.073406,0,truth


## Compute temporal aggregates (mean/max/var) per video

In [9]:
# compute aggregates (mean/max/std) per video pt.2

grouped = dataset.groupby(['video_id', 'face_id', 'label'], sort = False)

# create temporal diff for AU columns
for c in au_cols:
    dataset[f"dau_{c}"] = grouped[c].diff().fillna(0)
    dataset[f"adau_{c}"] = grouped[f"dau_{c}"].transform(np.abs)

# create temporal diff for gaze/pose columns
for c in au_cols:
    dataset[f"dgp_{c}"] = grouped[c].diff().fillna(0)
    dataset[f"adgp_{c}"] = grouped[f"dgp_{c}"].transform(np.abs)

dataset

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,dgp_AU20_r,adgp_AU20_r,dgp_AU23_r,adgp_AU23_r,dgp_AU25_r,adgp_AU25_r,dgp_AU26_r,adgp_AU26_r,dgp_AU45_r,adgp_AU45_r
0,1,0,0.000,0.98,1,-0.265343,0.270043,-0.925565,-0.394123,0.281717,...,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2,0,0.033,0.98,1,-0.269314,0.280146,-0.921405,-0.399485,0.291082,...,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.07,0.07
2,3,0,0.067,0.98,1,-0.276762,0.281786,-0.918694,-0.402257,0.294677,...,0.0,0.0,0.00,0.00,0.05,0.05,0.00,0.00,0.04,0.04
3,4,0,0.100,0.98,1,-0.267571,0.284353,-0.920624,-0.390941,0.297649,...,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,-0.04,0.04
4,5,0,0.133,0.98,1,-0.251324,0.286230,-0.924613,-0.374932,0.299439,...,0.0,0.0,0.00,0.00,0.02,0.02,0.00,0.00,0.01,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91331,536,0,19.815,0.98,1,0.399038,0.150437,-0.904509,0.234647,0.184356,...,0.0,0.0,-0.19,0.19,0.88,0.88,-0.49,0.49,0.08,0.08
91332,537,0,19.852,0.98,1,0.394478,0.167984,-0.903420,0.232950,0.200715,...,0.0,0.0,-0.25,0.25,0.79,0.79,-0.44,0.44,0.09,0.09
91333,538,0,19.889,0.98,1,0.392632,0.175124,-0.902869,0.232472,0.207146,...,0.0,0.0,-0.30,0.30,-0.49,0.49,-0.92,0.92,0.05,0.05
91334,539,0,19.926,0.98,1,0.402144,0.140985,-0.904657,0.236606,0.176743,...,0.0,0.0,-0.09,0.09,-0.45,0.45,-0.05,0.05,-0.10,0.10


In [10]:
#dataset.columns.values

In [11]:
# keep only diff_au, adiff_au, diff_gp & adiff_gp columns

diff_au_cols = [c for c in dataset.columns if c.startswith('dau_')]

adiff_au_cols = [c for c in dataset.columns if c.startswith('adau_')]

diff_gp_cols = [c for c in dataset.columns if c.startswith('dgp_')]

adiff_gp_cols = [c for c in dataset.columns if c.startswith('adgp_')]

#final_columns = au_cols + pose_cols + pose_cols
#final_columns

In [12]:
# final step: group by & aggregate

# AU diffs
dau_mean = dataset.groupby(['video_id', 'face_id', 'label'])[diff_au_cols].mean().add_prefix("dau_mean_")
dau_min = dataset.groupby(['video_id', 'face_id', 'label'])[diff_au_cols].min().add_prefix("dau_min_")
dau_max = dataset.groupby(['video_id', 'face_id', 'label'])[diff_au_cols].max().add_prefix("dau_max_")
dau_var = dataset.groupby(['video_id', 'face_id', 'label'])[diff_au_cols].var().add_prefix("dau_var_")

# AU absolute diffs
adau_mean = dataset.groupby(['video_id', 'face_id', 'label'])[adiff_au_cols].mean().add_prefix("adau_mean_")
adau_min = dataset.groupby(['video_id', 'face_id', 'label'])[adiff_au_cols].min().add_prefix("adau_min_")
adau_max = dataset.groupby(['video_id', 'face_id', 'label'])[adiff_au_cols].max().add_prefix("adau_max_")
adau_var = dataset.groupby(['video_id', 'face_id', 'label'])[adiff_au_cols].var().add_prefix("adau_var_")

# gaze & pose diffs
dgp_mean = dataset.groupby(['video_id', 'face_id', 'label'])[diff_gp_cols].mean().add_prefix("dgp_mean_")
dgp_min = dataset.groupby(['video_id', 'face_id', 'label'])[diff_gp_cols].min().add_prefix("dgp_min_")
dgp_max = dataset.groupby(['video_id', 'face_id', 'label'])[diff_gp_cols].max().add_prefix("dgp_max_")
dgp_var = dataset.groupby(['video_id', 'face_id', 'label'])[diff_gp_cols].var().add_prefix("dgp_var_")

# gaze & pose absolute diffs
adgp_mean = dataset.groupby(['video_id', 'face_id', 'label'])[adiff_gp_cols].mean().add_prefix("adgp_mean_")
adgp_min = dataset.groupby(['video_id', 'face_id', 'label'])[adiff_gp_cols].min().add_prefix("adgp_min_")
adgp_max = dataset.groupby(['video_id', 'face_id', 'label'])[adiff_gp_cols].max().add_prefix("adgp_max_")
adgp_var = dataset.groupby(['video_id', 'face_id', 'label'])[adiff_gp_cols].var().add_prefix("adgp_var_")

In [13]:
# union all aggregates in one table at last

agg_blocks = [
    dau_mean, dau_min, dau_max, dau_var, # AU diffs
    adau_mean, adau_min, adau_max, adau_var,  # AU absolute diffs
    dgp_mean, dgp_min, dgp_max, dgp_var,   # gaze & pose diffs
    adgp_mean, adgp_min, adgp_max, adgp_var # gaze & pose absolute diffs
]

for agg in agg_blocks:
    if agg.empty:
        continue
    data_temp = data_temp.merge(agg.reset_index(), on = ['video_id', 'face_id', 'label'], how = 'left')

# Output Files

#### ---> From now on we have one row per video with AU, Pose, Gaze mean/max/std & label

In [14]:
data_temp.columns

Index(['video_id', 'face_id', 'label', 'dau_mean_dau_AU01_r',
       'dau_mean_dau_AU02_r', 'dau_mean_dau_AU04_r', 'dau_mean_dau_AU05_r',
       'dau_mean_dau_AU06_r', 'dau_mean_dau_AU07_r', 'dau_mean_dau_AU09_r',
       ...
       'adgp_var_adgp_AU10_r', 'adgp_var_adgp_AU12_r', 'adgp_var_adgp_AU14_r',
       'adgp_var_adgp_AU15_r', 'adgp_var_adgp_AU17_r', 'adgp_var_adgp_AU20_r',
       'adgp_var_adgp_AU23_r', 'adgp_var_adgp_AU25_r', 'adgp_var_adgp_AU26_r',
       'adgp_var_adgp_AU45_r'],
      dtype='object', length=275)

In [15]:
data_temp.to_pickle('kaggle_aggr_temporal_v2.pkl')

In [16]:
dataset_st_mean.to_pickle('kaggle_aggr_mean_v2.pkl')
dataset_st_max.to_pickle('kaggle_aggr_max_v2.pkl')
dataset_st_std.to_pickle('kaggle_aggr_std_v2.pkl')